# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [17]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='b'].call))
print(sum(data[data.race=='w'].call))
print((data[data.race=='w']).call.count())
print((data[data.race=='b']).call.count())
print(data.head())

157.0
235.0
2435
2435
  id ad  education  ofjobs  yearsexp  honors  volunteer  military  empholes  \
0  b  1          4       2         6       0          0         0         1   
1  b  1          3       3         6       0          1         1         0   
2  b  1          4       1         6       0          0         0         0   
3  b  1          3       4         6       0          1         0         1   
4  b  1          3       3        22       0          0         0         0   

   occupspecific    ...      compreq  orgreq  manuf  transcom  bankreal trade  \
0             17    ...          1.0     0.0    1.0       0.0       0.0   0.0   
1            316    ...          1.0     0.0    1.0       0.0       0.0   0.0   
2             19    ...          1.0     0.0    1.0       0.0       0.0   0.0   
3            313    ...          1.0     0.0    1.0       0.0       0.0   0.0   
4            313    ...          1.0     1.0    0.0       0.0       0.0   0.0   

  busservice oth

In [10]:
def permutation_sample(data_1, data_2):
    #PERMUTATING THE DATA
    permuted_data = np.random.permutation(np.concatenate((data_1, data_2)))
    return permuted_data[:len(data_1)], permuted_data[len(data_1):]

def draw_perm_reps(data_1, data_2, func, size=1):
    #Generate multiple permutation replicates.

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates


def bootstrap_replicate_1d(data, func):
    #Generate bootstrap replicate of 1D data.
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample) 

def draw_bs_reps(data, func, size=1):
    #Draw bootstrap replicates

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates

## What test is appropriate for this problem? Does CLT apply?

    A permutation test is most appropriate for this problem.  There are two charateristics of interest, race and call back.  We state the null hypothese as race has no affect on whether a person is called back or not.  So we randomly assign whether a person was called back or not to white and black sounding names and determine the ratio of callback rate for black vs white sounding names, then calculate the P-value of achieving a value atleast as extreme as what we recorded 157/2435 : 235/2435.  Yes clt applies because there the data set is suficiently large (>30)
    
## What are the null and alternate hypotheses?

    Null hypothesis: Race has no effect on whether a person is called back or not for a position.
    Alternate hypothesis: Race does have an effect on whether a person is called back ornot for a position

In [16]:
#FUNCTION FOR PERMUTATIONS
def diff_of_ratio(data_1,data_2):
    len_1=len(data_1)
    sum_1=sum(data_1)
    len_2=len(data_2)
    sum_2=sum(data_2)
    #print(len_1,len_2,sum_1,sum_2)
    
    ratio = ((sum_1/len_1)/(sum_2/len_2))
    #print(len_1,len_2,sum_1,sum_2,ratio)
    return ratio

#CREATING ARRAYS FOR BLACK AND WHITE SOUNDING NAMES
white = np.array(data[data.race=='w'].call)
black = np.array(data[data.race=='b'].call)

#CALCULATING EMPIRICAL DIFFERNECE IN RATIO BETWEEN BLACK AND WHITE SOUNDING NAMES
empirical_diff_of_ratio =diff_of_ratio(white,black)
#print(empirical_diff_of_ratio)

#TAKING 10000 PERMUTATION UNDER THE NULL HYPOTHESE
race_perm_reps=draw_perm_reps(white,black,diff_of_ratio,10000)

#CALCULATING P VALUE
p=sum(race_perm_reps>=empirical_diff_of_ratio)/len(race_perm_reps)

#PRINTING P VALUE
print('p = ', p)

p =  0.0001


In [14]:
#MAKING BOOTSTRAP REPLICATES OF MEAN FOR WHITE AND BLACK DATASETS
white_bs_reps=draw_bs_reps(white,np.mean,10000)
black_bs_reps=draw_bs_reps(black,np.mean,10000)

#CREATING ARRAY OF THE RATIO
ratio_white_to_black=white_bs_reps/black_bs_reps

#CALCULATING SER
sem=np.std(ratio_white_to_black)/np.sqrt(len(ratio_white_to_black))
print("Standard error of the ratio (SER): "+str(sem))

#CALCULATING CONFIDENCE INTERVALS
print("95% confidence interval in ratio :" +str(np.percentile(ratio_white_to_black,[2.5,97.5])))

Standard error of the ratio (SER): 0.00150967127932
95% confidence interval in ratio :[ 1.23563144  1.8240204 ]


## Compute margin of error, confidence interval, and p-value.

    We calculated a p value = 0.0001 indicating that it is very unlikely that we observed a ration of call backs of 1.49681528662 in favor of white sounding names, meaning white sounding names are 50% more likely to be called back.  we calculated a standard error of 0.00150967127932 and a 95% confidence interval of [ 1.23563144  1.8240204 ]
    
## Write a story describing the statistical significance in the context or the original problem.

    using the above calculated statistics it is clear that this data shows us that black sounding names are less likely to be called back for an interview compared to white sounding names, meaning race is a factor in that decision.  This fits in with the discrimintation apparent in the US workforce and can be used to explain also why black people are often payed less than white people for the same job
    
## Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

    No, my analysis simply indicates that race/name is A factor in callback success.  We see that when all other aspects of the resume are kept constant, white sounding names are more likely to be called back.  To determine the most import factor in callback success one would need to repeat this experiment with each aspect of the resume, ie keeping name and all other aspects constant and changing years of experience or changing military experience to determine which factor creates the biggest change in callba

